<a href="https://colab.research.google.com/github/jesvin1/Basics/blob/master/PCR_MaxPain_high_OI_change_in_OI_nifty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
from datetime import datetime

# Colunn at which strike price is listed in NSE option chain table
strike_price_column_index = 11

# Encapuslate NSE option data and function
class OptionChain:

    # static variable to hold current running expiry date
    expiry = ''

    # Common Utility to find maxinum value in the column, its index and then return the strike price with respect to index value
    def find_max_strike_price(self, df, option_type, column_name):

        # Covert "-" as 0 so that all data can be treated as integer
        temp_df = df[option_type].replace("-", "0")
        # Delete the last row which will have summation of all the data
        temp_df = temp_df[:-1]
        # Set the specific column as integer (by default it is string)
        temp_df[column_name]=temp_df[column_name].astype(int)
        # Dind the index value where the max value exists
        max_at_index = temp_df[column_name].idxmax()
        # Return the strike price which is available in the index value
        return(int(df.iloc[max_at_index, strike_price_column_index]))

    # Get strike prices where max OI /change in OI for CE and PE
    def get_max_OI_data(self, symbol, expiry):

        url ="https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=" + symbol + "&date" + expiry
        header = {
          "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
          "X-Requested-With": "XMLHttpRequest"
        }

        # Pull NSE option chain
        r = requests.get(url, headers=header)
        # Convert html page as Table and read the first table which has option data
        df = pd.read_html(r.text)[1]

        # Get all max OI data and store to local object variables
        self.max_high_oi_ce = self.find_max_strike_price(df, "CALLS", "OI")
        self.max_change_oi_ce = self.find_max_strike_price(df, "CALLS", "Chng in OI")
        self.max_high_oi_pe = self.find_max_strike_price(df, "PUTS", "OI")
        self.max_change_oi_pe = self.find_max_strike_price(df, "PUTS", "Chng in OI")

    # Constructor for OptionChain which will run for every script like Nifty and Banknifty
    def __init__(self, symbol):

        # Store the symbol
        self.symbol = symbol
        # Find out the expiry for which we need to pull the details

        if not OptionChain.expiry: # If expiry is not yet found
            # List the expiry details and read the first expiry
            base_url = 'https://www.capitalzone.in/test.php?symbol=' + symbol
            page_output = str(requests.get(base_url).content)
            expiry_list = page_output.split(",")
            OptionChain.expiry = expiry_list[0].split("\"")[1]
            expirydate = datetime.strptime(OptionChain.expiry, '%d%b%Y').date()
            todaydate = datetime.today().date()

            # If today is greater than expiry, then it is expired. Choose the next expiry in the list
            if todaydate > expirydate:
                NSEOption.expiry = expiry_list[1].split("\"")[1]

        # Form the actual URL from which we can pull PCR and max pain
        base_url = 'https://www.capitalzone.in/test.php?symbol=' + symbol + "&expiry=" + OptionChain.expiry

        # Load the page and sent to HTML parse
        page_output = str(requests.get(base_url).content)
        # Locate the string where max pain string is available
        loc = page_output.find("\"max_pain\"")
        data = page_output[loc:].split("\"")
        # Get max pain and convert from float to integer
        self.max_pain = int(float(data[3]))
        # Get max pain and convert from string to float
        self.pcr = float(data[11])
        self.get_max_OI_data(symbol, OptionChain.expiry)

    # Tracing utility to display elements of this class if needed (For debugging purpose)
    def display_all(self):

        print(self.max_pain, self.pcr);

In [0]:
import requests

In [0]:
url ="https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=nifty"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36',
}

# Pull NSE option chain
#r = requests.get(url, headers=header)
# Convert html page as Table and read the first table which has option data
#df = pd.read_html(r.text)[1]

In [0]:
try:
  response = requests.get(url, timeout=10, headers=HEADERS)

  # If the response was successful, no Exception will be raised
  response.raise_for_status()

#except HTTPError as http_err:
    #print(f'HTTP error occurred: {http_err}')  # Python 3.6
except Exception as err:
    print(f'Other error occurred: {err}')  # Python 3.6
else:
    print('Success!')

Other error occurred: HTTPSConnectionPool(host='www.nseindia.com', port=443): Read timed out. (read timeout=10)


In [2]:

import requests
import pandas as pd
from bs4 import BeautifulSoup



def get_option_chain(symbol):

    if symbol == 'NIFTY':
        Base_url =('https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?'
                   'symbolCode=-10003&symbol=NIFTY&symbol=NIFTY&instrument=OPTIDX&date=-&segmentLink=17&segmentLink=17')
    elif symbol == 'BANKNIFTY':
        Base_url =('https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?'
                   'symbolCode=-9999&symbol=BANKNIFTY&symbol=BANKNIFTY&instrument=OPTIDX&date=-&segmentLink=17&segmentLink=17')
    else:
        Base_url = ('https://www.nseindia.com//marketinfo/sym_map/symbolMapping.jsp?symbol={}&'
                    'instrument=OPTSTK&date=-&segmentLink=17'.format(symbol))
    try:
        page = requests.get(Base_url,timeout=10)
    except Exception as e:
        print ("Exception {} in getting data  for symbol {}".format(e, symbol))

    #print(page.status_code)
    #print(page.content)

    soup = BeautifulSoup(page.content, 'html.parser')
    # print (soup.prettify())

    table_it = soup.find_all("div", {"class": "opttbldata"})
    table_cls_1 = soup.find_all(id='octable')

    # print (table_it)
    # print (table_cls_1)

    col_list = []

    for mytable in table_cls_1:
        table_head = mytable.find('thead')

        try:
            row = table_head.find_all('tr')
            for tr in row:
                cols = tr.find_all('th')
                for th in cols:
                    er = th.text
                    ee = er.encode('utf-8')
                    col_list.append(ee)
        except Exception as e:
            print ("No thead", e)

    col_list_fnl = [e for e in col_list if e not in ('CALLS', 'PUTS', 'Chart', '\xc2\xa0')]

    # print (col_list_fnl)


    table_cls_2 = soup.find(id='octable')
    all_trs = table_cls_2.find_all('tr')
    req_rows = table_cls_2.find_all('tr')

    new_table = pd.DataFrame(index=range(0, len(req_rows)-3), columns=col_list_fnl)

    row_marker = 0

    for row_number, tr_nos in enumerate(req_rows):
        if row_number < 1 or row_number == len(req_rows)-1:
            continue
        td_columns = tr_nos.find_all('td')

        select_cols = td_columns[1:22]
        cols_h = range(0, len(select_cols))

        for nu, column in enumerate(select_cols):
            utf_string = column.get_text().strip('\n\r\t": ')
            tr = utf_string.encode('utf-8').replace(',', '')
            new_table.ix[row_number, [nu]] = tr

        row_marker += 1

    print (new_table)
    new_table.to_csv('Option_Chain_Table_{}.csv'.format(symbol))



#get_option_chain('WIPRO')
get_option_chain('NIFTY')
#get_option_chain('BANKNIFTY') 

Exception HTTPSConnectionPool(host='www.nseindia.com', port=443): Read timed out. (read timeout=10) in getting data  for symbol NIFTY


UnboundLocalError: ignored

In [9]:
!pip install BeautifulSoup
http://quantsnippets.blogspot.com/2014/01/get-option-dataspot-price-from-nse.html

ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [0]:
from urllib.request import urlopen 
#from BeautifulSoup import BeautifulSoup  
#import datetime  
#import pandas  
#import csv  
#import os  
#import re  
#import logging 

In [0]:
option_site = 'http://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=nifty'  


In [15]:
hdr = {'User-Agent': 'Web-Scraping',  
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',  
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',  
    'Accept-Encoding': 'none',  
    'Accept-Language': 'en-US,en;q=0.8',  
    'Connection': 'keep-alive'}  
       
html = urlopen(option_site).read()
print(html)

KeyboardInterrupt: ignored

In [20]:
import urllib.request
url = "http://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=nifty"
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
data_content = response.read()
print(data_content)

KeyboardInterrupt: ignored

In [0]:
from urllib.request import urlopen
content = urlopen("https://www.quora.com/")

In [19]:
print(content)

In [0]:
import requests
from bs4 import BeautifulSoup

# Get all get possible expiry date details for the given script
def get_expiry_from_option_chain (symbol):

    # Base url page for the symbole with default expiry date
    Base_url = "https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=" + symbol + "&date=-"

    # Load the page and sent to HTML parse
    page = requests.get(Base_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Locate where expiry date details are available
    locate_expiry_point = soup.find(id="date")
    # Convert as rows based on tag option
    expiry_rows = locate_expiry_point.find_all('option')

    index = 0
    expiry_list = []
    for each_row in expiry_rows:
        # skip first row as it does not have value
        if index <= 0:
            index = index + 1
            continue
        index = index + 1
        # Remove HTML tag and save to list
        expiry_list.append(BeautifulSoup(str(each_row), 'html.parser').get_text())

    # print(expiry_list)
    return expiry_list # return list

def get_strike_price_from_option_chain(symbol, expdate):

    Base_url = "https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=" + symbol + "&date=" + expdate

    page = requests.get(Base_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    table_cls_2 = soup.find(id="octable")
    req_row = table_cls_2.find_all('tr')

    strike_price_list = []

    for row_number, tr_nos in enumerate(req_row):

        # This ensures that we use only the rows with values
        if row_number <= 1 or row_number == len(req_row) - 1:
            continue

        td_columns = tr_nos.find_all('td')
        strike_price = int(float(BeautifulSoup(str(td_columns[11]), 'html.parser').get_text()))
        strike_price_list.append(strike_price)

    # print (strike_price_list)
    return strike_price_list

In [23]:
print(get_expiry_from_option_chain('nifty'))

ConnectionError: ignored